In [1]:
from fractions import Fraction
from functools import reduce 

def num_of_transients(m):
    if len(m) == 0:
        raise Exception("Can't get transient states of empty matrix")

    for r in range(len(m)):
        for c in range(len(m[r])):
            if m[r][c] != 0:
                # this is not an all-zero row, try next one
                break
        else:
            # has just finished looping over an empty row (i.e. w/o `break`)
            return r
    # reached end of table and didn't encounter all-zero row - no absorbing states
    raise Exception("Not a valid AMC matrix: no absorbing (terminal) states")


# decompose input matrix `m` on Q (t-by-t) and R (t-by-r) components
# `t` is the number of transient states
def decompose(m):
    t = num_of_transients(m)
    if t == 0:
        raise Exception("No transient states. At least initial state is needed.")

    Q = []
    for r in range(t):
        qRow = []
        for c in range(t):
            qRow.append(m[r][c])
        Q.append(qRow)
    if Q == []:
        raise Exception("Not a valid AMC matrix: no transient states")

    R = []
    for r in range(t):
        rRow = []
        for c in range(t, len(m[r])):
            rRow.append(m[r][c])
        R.append(rRow)
    if R == []:
        raise Exception("Not a valid AMC matrix: missing absorbing states")
    return Q, R

# return Identity matrix of size `t`
def identity(t):
    m = []
    for i in range(t):
        r = []
        for j in range(t):
            r.append(int(i == j))
        m.append(r)
    return m

# check if the matrix is zero
def isZero(m):
    for r in range(len(m)):
        for c in range(len(m[r])):
            if m[r][c] != 0:
                return False
    return True


# swap i,j rows/columns of a square matrix `m`
def swap(m, i, j):
    n = []
    s = len(m)

    if s != len(m[0]):
        raise Exception("Cannot swap non-square matrix")

    if i == j:
        # no need to swap
        return m

    for r in range(s):
        nRow = []
        tmpRow = m[r]
        if r == i:
            tmpRow = m[j]
        if r == j:
            tmpRow = m[i]
        for c in range(s):
            tmpEl = tmpRow[c]
            if c == i:
                tmpEl = tmpRow[j]
            if c == j:
                tmpEl = tmpRow[i]
            nRow.append(tmpEl)
        n.append(nRow)
    return n

# reorganize matrix so zero-rows go last (preserving zero rows order)
def sort(m):
    size = len(m)

    zero_row = -1
    for r in range(size):
        sum = 0
        for c in range(size):
            sum += m[r][c]
        if sum == 0:
            # we have found all-zero row, remember it
            zero_row = r
        if sum != 0 and zero_row > -1:
            # we have found non-zero row after all-zero row - swap these rows
            n = swap(m, r, zero_row)
            # and repeat from the begining
            return sort(n)
    #nothing to sort, return
    return m

# normalize matrix `m`
def normalize(m, use_fractions=False ):
    n = []
    for r in range(len(m)):
        sum = 0
        cols = len(m[r])
        for c in range(cols):
            sum += m[r][c]

        nRow = []

        if sum == 0:
            # all-zero row
            nRow = m[r]
        else:
            for c in range(cols):
                # FIXME it's strange but python 2.7 does not automatically convert decimals to floats
                if use_fractions:
                    nRow.append(Fraction(m[r][c], sum))
                else:
                    nRow.append(float(m[r][c])/sum)
        n.append(nRow)
    return n

# subtract two matrices
def subtract(i, q):
    if len(i) != len(i[0]) or len(q) != len(q[0]):
        raise Exception("non-square matrices")

    if len(i) != len(q) or len(i[0]) != len(q[0]):
        raise Exception("Cannot subtract matrices of different sizes")

    s = []
    for r in range(len(i)):
        sRow = []
        for c in range(len(i[r])):
            sRow.append(i[r][c] - q[r][c])
        s.append(sRow)
    return s

# multiply two matrices
def multiply(a, b):
    if a == [] or b == []:
        raise Exception("Cannot multiply empty matrices")

    if len(a[0]) != len(b):
        raise Exception("Cannot multiply matrices of incompatible sizes")

    m = []
    rows = len(a)
    cols = len(b[0])
    iters = len(a[0])

    for r in range(rows):
        mRow = []
        for c in range(cols):
            sum = 0
            for i in range(iters):
                sum += a[r][i]*b[i][c]
            mRow.append(sum)
        m.append(mRow)
    return m

# transpose matrix
def transposeMatrix(m):
    t = []
    for r in range(len(m)):
        tRow = []
        for c in range(len(m[r])):
            if c == r:
                tRow.append(m[r][c])
            else:
                tRow.append(m[c][r])
        t.append(tRow)
    return t

def getMatrixMinor(m,i,j):
    return [row[:j] + row[j+1:] for row in (m[:i]+m[i+1:])]

# matrix determinant
def getMatrixDeternminant(m):
    #base case for 2x2 matrix
    if len(m) == 2:
        return m[0][0]*m[1][1]-m[0][1]*m[1][0]

    d = 0
    for c in range(len(m)):
        d += ((-1)**c)*m[0][c]*getMatrixDeternminant(getMatrixMinor(m,0,c))

    return d

# matrix inversion
def getMatrixInverse(m):
    d = getMatrixDeternminant(m)

    if d == 0:
        raise Exception("Cannot get inverse of matrix with zero determinant")

    #special case for 2x2 matrix:
    if len(m) == 2:
        return [[m[1][1]/d, -1*m[0][1]/d],
                [-1*m[1][0]/d, m[0][0]/d]]

    #find matrix of cofactors
    cofactors = []
    for r in range(len(m)):
        cofactorRow = []
        for c in range(len(m)):
            minor = getMatrixMinor(m,r,c)
            cofactorRow.append(((-1)**(r+c)) * getMatrixDeternminant(minor))
        cofactors.append(cofactorRow)
    cofactors = transposeMatrix(cofactors)
    for r in range(len(cofactors)):
        for c in range(len(cofactors)):
            cofactors[r][c] = cofactors[r][c]/d
    return cofactors

In [2]:
def calculate_b(m,use_fractions=False):
        # B = (I-Q)^-1 * R
        m = sort(m)
        n = normalize(m,use_fractions=use_fractions)
        (q, r) = decompose(n)
        i = identity(len(q))
        s = subtract(i, q)
        v = getMatrixInverse(s)
        b = multiply(v, r)
        return b
    
def lcm(a, b):
        if a > b:
            greater = a
        else:
            greater = b

        while True:
            if greater % a == 0 and greater % b == 0:
                lcm = greater
                break
            greater += 1

        return lcm

def get_lcm_for(l):
    return reduce(lambda x, y: lcm(x, y), l)

def convert_to_lcd(probs):
    ret = []
    least_common_multiple = get_lcm_for([f.denominator for f in probs])
    for f in probs:
        if f.denominator != least_common_multiple:
            ret.append((least_common_multiple * f.numerator) // f.denominator)
        else:
            ret.append(f.numerator)
    ret.append(least_common_multiple)
    return ret

def markov_probabilities(m):
    probs = calculate_b(m, use_fractions=True)[0]
    return convert_to_lcd(probs)

In [19]:
from collections import defaultdict
from fractions import Fraction


def prod(arr):
    res = 1
    for elem in arr:
        res *= elem
    return res

class solver2():
    def __init__(self, m):
        self.m = m
        self.isTerminal = [True if sum(i) == 0 else False for i in self.m]
        self.terminals = defaultdict(lambda: [])
        self.handeled = [] 
        
    def visit(self, curIdx, curProb, visited = []):
            if visited.count(curIdx) > 0:
                if visited.count(curIdx) > 1:
                    return

                cicle = visited[visited.index(curIdx):] + [curIdx]
                if cicle in self.handeled:
                    return

                self.handeled.append(cicle)
                cicleProb = prod(curProb[visited.index(curIdx):])
                for i in range(len(self.m[curIdx])):
                    if self.m[curIdx][i] != 0:
                        if self.isTerminal[i]:
                            termProb = prod(curProb[:visited.index(curIdx)] 
                                              + [Fraction(self.m[curIdx][i], sum(self.m[curIdx]))])
                            prob_i = termProb * Fraction(cicleProb.denominator, cicleProb.denominator - cicleProb.numerator)
                            prob_i -= termProb
                            #geometricSum(r=cicleProb, a=termProb)
                            self.terminals[i] += [prob_i]
                        else:
                            prob_i = curProb + [Fraction(self.m[curIdx][i], sum(self.m[curIdx]))]
                            self.visit(i, prob_i, visited+[curIdx])
                return
            
            if self.isTerminal[curIdx]:
                self.terminals[curIdx] += [prod(curProb)]
                return

            for i in range(0,len(self.m[curIdx])):
                if self.m[curIdx][i] != 0:
                    prob_i = curProb + [Fraction(self.m[curIdx][i], sum(self.m[curIdx]))]
                    self.visit(i, prob_i, visited+[curIdx])
            return 
        
    def getRes(self):
        self.visit(0, [], [])
        res = []
        for i in range(len(self.isTerminal)):
            if self.isTerminal[i]:
                if i in self.terminals.keys():
                    res.append(sum(self.terminals[i]))
                else:
                    res.append(Fraction(0,1))
        return res

In [20]:
m = [[0, 1, 0, 0, 0, 1], [4, 0, 0, 3, 2, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0],
             [0, 0, 0, 0, 0, 0]]

# tansitionMatrix = [[0, 2, 1, 0, 0],
#                    [0, 0, 0, 3, 4],
#                    [0, 0, 0, 0, 0],
#                    [0, 0, 0, 0, 0],
#                    [0, 0, 0, 0, 0]]

# tansitionMatrix = [[0, 1, 0, 0, 0, 1],
#                    [4, 0, 0, 3, 2, 0],
#                    [0, 0, 0, 0, 0, 0],
#                    [0, 0, 0, 0, 0, 0],
#                    [0, 0, 0, 0, 0, 0],
#                    [0, 0, 0, 0, 0, 0]]

tansitionMatrix = [[0, 0, 1], 
                   [0, 0, 0], 
                   [0, 0, 0]] 

# tansitionMatrix = [[1, 1, 1, 1], 
#                    [1, 1, 1, 1], 
#                    [1, 1, 1, 1],
#                    [0, 0, 0, 0]] 

# tansitionMatrix = [[0, 1, 0, 1], 
#                    [0, 0, 1, 0], 
#                    [0, 1, 0, 0],
#                    [0, 0, 0, 0]] 

# m = [[0, 0, 0, 0, 3, 5, 0, 0, 0, 2],
#      [0, 0, 4, 0, 0, 0, 1, 0, 0, 0],
#      [0, 0, 0, 4, 4, 0, 0, 0, 1, 1],
#      [13, 0, 0, 0, 0, 0, 2, 0, 0, 0],
#      [0, 1, 8, 7, 0, 0, 0, 1, 3, 0],
#      [1, 7, 0, 0, 0, 0, 0, 2, 0, 0],
#      [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
#      [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
#      [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
#      [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

markov_probabilities(m)

[0, 3, 2, 9, 14]

In [21]:
s = solver2(m)
probs = s.getRes()
convert_to_lcd(probs)

[0, 3, 2, 9, 14]

In [18]:
probs

[Fraction(0, 1), Fraction(5, 24), Fraction(5, 36), Fraction(9, 14)]